In [1]:
#run flask in Colab
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install flask_restful
!pip install flask_cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken # needed for OpenAI embeddings
!pip install sentence_transformers
!pip install -qU transformers accelerate einops langchain wikipedia xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#first setup LLM


In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Split the extracted text into chunks so we can avoid hitting the token limit of the LLM 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)# Split the extracted text into chunks so we can avoid hitting the token limit of the LLM 


In [7]:
from torch import cuda, bfloat16
import transformers

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

model = transformers.AutoModelForCausalLM.from_pretrained(
    'mosaicml/mpt-7b-instruct',
    trust_remote_code=True,
    torch_dtype=bfloat16,
    max_seq_len=2048
)
model.eval()
model.to(device)

tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

# mtp-7b is trained to add "<|endoftext|>" at the end of generations
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    device=device,
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=64,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)


The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFor

In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
#test llm
print(llm_chain.predict(
    instruction="Explain to me the difference between nuclear fission and fusion."
).lstrip())

Nuclear Fission is a process that splits heavy atoms into smaller, lighter ones by releasing energy in the form of heat or radiation. Nuclear Fusion occurs when two light atomic nuclei are combined together to create one heavier nucleus which releases more energy than what was used for its creation (fusion reaction). The most common example of


In [11]:
# Load our Q+A chain
from langchain.chains.question_answering import load_qa_chain
chain2 = load_qa_chain(llm=llm, chain_type="stuff")

In [12]:
#then setup Flask

In [13]:
from flask import Flask, request
from flask_restful import Api, Resource
from flask_cors import CORS, cross_origin

import os
import threading

from pyngrok import ngrok, conf


In [14]:
class WebPage():
	url = ''
	content = ''
	title = ''
	
class QA():
	question = ''
	answer = ''
	references = []

allDocs = []

In [15]:
docsearch = None

In [16]:
class IngestDocument(Resource):
    response = {"status":200, "message":"success"}
    def addContent(self,content):
        global docsearch;
        texts = text_splitter.split_text(content)
        print("split to chunks", len(texts))
        if(docsearch == None):
          print("create new docsearch")
          docsearch = FAISS.from_texts(texts, hf)
        else:
          print("add to docsearch")
          docsearch.add_texts(texts)
        print("add content success")


    def post(self):
        global allDocs;
        page_data = request.get_json()
        url = page_data['url']
        content = page_data['content']
        title = page_data['title']

        doc = WebPage()
        doc.url = url
        doc.content = content
        doc.title = title
        allDocs.append(doc)
        print("now add content to FAISS", title)
        self.addContent(content)

        self.response['status'] = 200
        self.response['message'] = 'success'
        return self.response, 200

In [17]:
def runLocalTest():
  global docsearch;
  content = "I’ve observed thousands of founders and thought a lot about what it takes to make a huge amount of money or to create something important. Usually, people start off wanting the former and end up wanting the latter.\n\nHere are 13 thoughts about how to achieve such outlier success. Everything here is easier to do once you’ve already reached a baseline degree of success (through privilege or effort) and want to put in the work to turn that into outlier success. "

  texts = text_splitter.split_text(content)
  print("split to chunks", len(texts))
  if(docsearch == None):
    print("create new docsearch")
    docsearch = FAISS.from_texts(texts, hf)
  else:
    print("add to docsearch")
    docsearch.add_texts(texts)
    print("add content success")

  question = "how to make huge amount of money"
  refdocs = docsearch.similarity_search(question)
  print(refdocs)
  answer = chain2.run(input_documents=refdocs, question=question)
  print(answer)


In [26]:
query = "Give me some more tangible advice. What are 3 key steps I can take today to be more successful according to Sam Altman?"
docs = docsearch.similarity_search(query)
answer = chain2.run(input_documents=docs, question=query)
print(answer)

 Here's a few suggestions based off his book: 1.) Cultivate optimism 2.) Trust the Exponential 3.) Follow Your Curiosity


In [ ]:
runLocalTest()

In [19]:
class AskQuestion(Resource):
    response = {"status":200}

    def searchRefDocs(self, question):
        if(docsearch==None):
          docs=[]
        else:  
          docs = docsearch.similarity_search(question)
        
        #docs = ["docs1","doc2"]
        return docs
	
    def askQuestion(self, refdocs, question):
        answer = chain2.run(input_documents=refdocs, question=question)
        #answer="test answer"
        return answer;
		
    def post(self):
        question_data = request.get_json()
        question = question_data['question']
		
        refdocs = self.searchRefDocs(question)
        print("searchRefDocs found", len(refdocs))
        answer = self.askQuestion(refdocs, question)
        print("askQuestion returns", answer)

        q_response = {
          "question": question,
          "answer": answer,
          "refdocs": []
        }
        self.response['status']=200
        self.response['message']= q_response
        return self.response, 200

In [20]:
class DocumentList(Resource):
    response = {"status":200}
    def title(self,x):
        return x.title;

    def get(self):
        titles =   list(map(self.title, allDocs))

        self.response['status']=200
        self.response['message']= titles
        return self.response, 200
    

In [21]:
class ResetDocumentList(Resource):
    response = {"status":200}
    def title(self,x):
        return x.title;

    def get(self):
        global allDocs;
        global docsearch;
        allDocs=[]
        docsearch=None

        self.response['status']=200
        self.response['message']= "success"
        return self.response, 200

In [22]:
class HelloWorld(Resource):
    def get(self):
        return {'hello': 'world'}
    def post(self):
        return {'hello': 'post world'}



In [23]:
app = Flask(__name__)
api = Api(app)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

In [24]:
api.add_resource(IngestDocument, '/api/ingest')
api.add_resource(AskQuestion, '/api/qa')
api.add_resource(HelloWorld, '/api/hello')
api.add_resource(DocumentList, '/api/docs')
api.add_resource(ResetDocumentList, '/api/resetdocs')

In [25]:

conf.get_default().auth_token = 'xxxxxxx' #sign up free account from ngrok
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "https://036e-34-142-128-239.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
